# Data Preprocessing: Airbnb Instant Book Study

**Estimand:** Effect of enabling Instant Book on a listing's current average rating.


In [1]:
import pandas as pd
import numpy as np
import re
import json
from pathlib import Path

pd.set_option('display.max_columns', 50)


In [2]:
# Set up paths
PROJECT_ROOT = Path("..").resolve()
RAW_DATA_PATH = PROJECT_ROOT / "data" / "raw" / "listings.csv.gz"
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed"
PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

# Load raw data
df_raw = pd.read_csv(RAW_DATA_PATH, compression='gzip', low_memory=False)
print(f"Loaded {len(df_raw)} listings with {len(df_raw.columns)} columns")


Loaded 7780 listings with 79 columns


In [3]:
df_raw['host_location'].value_counts()

host_location
San Francisco, CA    4465
New York, NY          214
Princeville, HI        87
Calgary, Canada        85
Los Angeles, CA        83
                     ... 
Union City, CA          1
Oakville, Canada        1
Manor, TX               1
Lahaina, HI             1
Dublin, CA              1
Name: count, Length: 227, dtype: int64

In [4]:
df_raw.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [5]:
df_raw.columns
also_needed = ['id', 'last_scraped', 'calendar_last_scraped','first_review', 'number_of_reviews']

covs_v1 = ['instant_bookable','neighbourhood_cleansed', 'host_listings_count', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'beds', 'amenities', 'bathrooms']

variable_of_interest = ['review_scores_rating']

COLS = also_needed + covs_v1 + variable_of_interest

In [6]:
# Select columns for analysis
df = df_raw[COLS].copy()
print(f"Selected {len(COLS)} columns")
df.info()


Selected 18 columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7780 entries, 0 to 7779
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      7780 non-null   int64  
 1   last_scraped            7780 non-null   object 
 2   calendar_last_scraped   7780 non-null   object 
 3   first_review            5910 non-null   object 
 4   number_of_reviews       7780 non-null   int64  
 5   instant_bookable        7780 non-null   object 
 6   neighbourhood_cleansed  7780 non-null   object 
 7   host_listings_count     7779 non-null   float64
 8   latitude                7780 non-null   float64
 9   longitude               7780 non-null   float64
 10  property_type           7780 non-null   object 
 11  room_type               7780 non-null   object 
 12  accommodates            7780 non-null   int64  
 13  bedrooms                7291 non-null   float64
 14  beds                

## Data Cleaning


In [7]:
#Clean instant_bookable
def clean_instant_bookable(x):
    if pd.isna(x):
        return np.nan
    return 1 if x == 't' else 0

df['instant_bookable'] = df['instant_bookable'].apply(clean_instant_bookable)
# Rename to treatment
df.rename(columns={'instant_bookable': 'treatment'}, inplace=True)

df['treatment'].value_counts()


treatment
0    5419
1    2361
Name: count, dtype: int64

In [8]:
# Count amenities from JSON list
def count_amenities(x):
    if pd.isna(x):
        return 0
    try:
        return len(json.loads(x))
    except:
        return len(str(x).split(','))

df['amenities_count'] = df['amenities'].apply(count_amenities)
print("Amenities count stats:")
print(df['amenities_count'].describe())


Amenities count stats:
count    7780.000000
mean       34.111825
std        15.320280
min         0.000000
25%        22.000000
50%        34.000000
75%        45.000000
max        91.000000
Name: amenities_count, dtype: float64


In [9]:
# Compute listing age in months
df['first_review'] = pd.to_datetime(df['first_review'], errors='coerce')

df['calendar_last_scraped'] = pd.to_datetime(df['calendar_last_scraped'], errors='coerce')

print(365.25 / 12)

df['listing_age_months'] = ((df['calendar_last_scraped'] - df['first_review']).dt.days / 30.44).round(1)

df.loc[df['listing_age_months'] < 0, 'listing_age_months'] = np.nan

print("Listing age (months) stats:")
print(df['listing_age_months'].describe())


30.4375
Listing age (months) stats:
count    5910.000000
mean       56.931607
std        43.889346
min         0.100000
25%        20.300000
50%        44.300000
75%        88.475000
max       196.000000
Name: listing_age_months, dtype: float64


## Handle Missing Values & Create Final Dataset


In [10]:
# Check missing values

analysis_vars = ['treatment','review_scores_rating','neighbourhood_cleansed', 'host_listings_count', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'beds', 'bathrooms','amenities_count', 'listing_age_months']


print("Missing values:")
for col in analysis_vars:
    n_miss = df[col].isna().sum()
    pct = 100 * n_miss / len(df)
    print(f"  {col}: {n_miss} ({pct:.1f}%)")


Missing values:
  treatment: 0 (0.0%)
  review_scores_rating: 1870 (24.0%)
  neighbourhood_cleansed: 0 (0.0%)
  host_listings_count: 1 (0.0%)
  latitude: 0 (0.0%)
  longitude: 0 (0.0%)
  property_type: 0 (0.0%)
  room_type: 0 (0.0%)
  accommodates: 0 (0.0%)
  bedrooms: 489 (6.3%)
  beds: 1958 (25.2%)
  bathrooms: 1934 (24.9%)
  amenities_count: 0 (0.0%)
  listing_age_months: 1870 (24.0%)


In [11]:
# Complete case analysis. Drop na values
print(f"Rows before: {len(df)}")

df_clean = df.dropna(subset=analysis_vars).copy()
print(f"Rows after complete case: {len(df_clean)}")

# Create final dataset
df_final = df_clean[['id'] + analysis_vars + ['number_of_reviews']].copy()

df_final = df_final.rename(columns={
    'neighbourhood_cleansed': 'neighbourhood',
    'review_scores_rating': 'rating'
})
df_final.head()


Rows before: 7780
Rows after complete case: 4647


,id,treatment,rating,neighbourhood,host_listings_count,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,bathrooms,amenities_count,listing_age_months,number_of_reviews
0,958,0,4.89,Western Addition,1.0,37.77028,-122.43317,Entire serviced apartment,Entire home/apt,3,1.0,2.0,1.0,53,193.3,496
1,5858,0,4.87,Bernal Heights,2.0,37.74474,-122.42089,Entire rental unit,Entire home/apt,4,2.0,2.0,2.0,17,196.0,105
2,8014,0,4.77,Outer Mission,3.0,37.73077,-122.44827,Private room in home,Private room,1,1.0,1.0,2.0,31,156.9,90
3,8142,0,4.70,Haight Ashbury,20.0,37.76555,-122.45213,Private room in rental unit,Private room,1,3.0,1.0,4.0,21,131.8,10
4,8339,0,4.86,Western Addition,1.0,37.77377,-122.43614,Entire condo,Entire home/apt,2,1.0,1.0,1.5,38,184.6,25


In [12]:
# Summary statistics
print(f"\nTotal listings: {len(df_final)}")
print(f"Treated (Instant Book=1): {df_final['treatment'].sum()} ({100*df_final['treatment'].mean():.1f}%)")
print(f"Control (Instant Book=0): {len(df_final) - df_final['treatment'].sum()} ({100*(1-df_final['treatment'].mean()):.1f}%)")
print(f"\nRating mean: {df_final['rating'].mean():.2f}, std: {df_final['rating'].std():.2f}")
print(f"Neighbourhoods: {df_final['neighbourhood'].nunique()}")
print(f"Room types: {df_final['room_type'].unique().tolist()}")



Total listings: 4647
Treated (Instant Book=1): 1176 (25.3%)
Control (Instant Book=0): 3471 (74.7%)

Rating mean: 4.80, std: 0.36
Neighbourhoods: 36
Room types: ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']


In [13]:
# Save processed data
output_path = PROCESSED_PATH / "listings_clean.parquet"
df_final.to_parquet(output_path, index=False)
print(f"Saved to: {output_path}")

# Verify
df_check = pd.read_parquet(output_path)
print(f"Verified: {df_check.shape}")


Saved to: /Users/william/Code/Project Stats209/data/processed/listings_clean.parquet
Verified: (4647, 16)
